In [0]:
dfc = spark.read.csv(path='dbfs:/FileStore/data/customers.csv')
display(dfc)

_c0,_c1,_c2,_c3
Name,Address,Email,Phone
John Doe,123 Main St,john.doe@example.com,1234567890
Jane Smith,456 Elm St,jane.smith@example.com,2345678901
Michael Johnson,789 Oak Ave,michael.johnson@example.com,3456789012
Emily Brown,101 Pine St,emily.brown@example.com,4567890123
John Doe,123 Main St,john.doe2@example.com,8521469589
Jane Smith,456 Elm St,jane.smith2@example.com,5678901234
David Wilson,222 Maple Rd,david.wilson@example.com,6789012345
Sarah Davis,333 Birch Ave,sarah.davis@example.com,7890123456
Michael Johnson,789 Oak Ave,michael.johnson2@example.com,9012345678


# Step 1: Loading the Customer Data

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Defining the schema explicitly based on CSV file
schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Address", StringType(), True),
    StructField("Email", StringType(), True),
    StructField("Phone", StringType(), True)
])

df_customers = spark.read.csv('dbfs:/FileStore/data/customers.csv', header=True, schema=schema)

# Show the loaded data to verify schema and column names
df_customers.show()

+---------------+--------------+--------------------+----------+
|           Name|       Address|               Email|     Phone|
+---------------+--------------+--------------------+----------+
|       John Doe|   123 Main St|john.doe@example.com|1234567890|
|     Jane Smith|    456 Elm St|jane.smith@exampl...|2345678901|
|Michael Johnson|   789 Oak Ave|michael.johnson@e...|3456789012|
|    Emily Brown|   101 Pine St|emily.brown@examp...|4567890123|
|       John Doe|   123 Main St|john.doe2@example...|8521469589|
|     Jane Smith|    456 Elm St|jane.smith2@examp...|5678901234|
|   David Wilson|  222 Maple Rd|david.wilson@exam...|6789012345|
|    Sarah Davis| 333 Birch Ave|sarah.davis@examp...|7890123456|
|Michael Johnson|   789 Oak Ave|michael.johnson2@...|9012345678|
|    Emma Miller|444 Cedar Blvd|emma.miller@examp...| 123456789|
+---------------+--------------+--------------------+----------+



# Step 2: Identifying and Assigning Master IDs to Duplicates

In [0]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

# Step 2: Identify duplicates based on Name and Address
window_spec = Window.partitionBy("name", "address").orderBy("name")
df_with_duplicates = df_customers.withColumn("MasterID", row_number().over(window_spec))

# Show the data with Master IDs assigned
df_with_duplicates.show()

+---------------+--------------+--------------------+----------+--------+
|           Name|       Address|               Email|     Phone|MasterID|
+---------------+--------------+--------------------+----------+--------+
|   David Wilson|  222 Maple Rd|david.wilson@exam...|6789012345|       1|
|    Emily Brown|   101 Pine St|emily.brown@examp...|4567890123|       1|
|    Emma Miller|444 Cedar Blvd|emma.miller@examp...| 123456789|       1|
|     Jane Smith|    456 Elm St|jane.smith@exampl...|2345678901|       1|
|     Jane Smith|    456 Elm St|jane.smith2@examp...|5678901234|       2|
|       John Doe|   123 Main St|john.doe@example.com|1234567890|       1|
|       John Doe|   123 Main St|john.doe2@example...|8521469589|       2|
|Michael Johnson|   789 Oak Ave|michael.johnson@e...|3456789012|       1|
|Michael Johnson|   789 Oak Ave|michael.johnson2@...|9012345678|       2|
|    Sarah Davis| 333 Birch Ave|sarah.davis@examp...|7890123456|       1|
+---------------+--------------+------

# Step 3: Saving the Enriched DataFrame

In [0]:
# Step 3: Save the enriched DataFrame back to DBFS or Azure SQL Database
# Save as CSV back to DBFS
df_with_duplicates.write.csv('dbfs:/FileStore/data/customers.csv', header=True, mode='overwrite')

print("Enriched customer data saved.")

Enriched customer data saved.


# Step 4: Reloading and Verifying Saved Data

In [0]:
enriched_customers_df = spark.read.csv('dbfs:/FileStore/data/customers.csv', header=True, inferSchema=True)

# Show the loaded DataFrame to verify
enriched_customers_df.show()


+---------------+--------------+--------------------+----------+--------+
|           Name|       Address|               Email|     Phone|MasterID|
+---------------+--------------+--------------------+----------+--------+
|   David Wilson|  222 Maple Rd|david.wilson@exam...|6789012345|       1|
|    Emily Brown|   101 Pine St|emily.brown@examp...|4567890123|       1|
|    Emma Miller|444 Cedar Blvd|emma.miller@examp...| 123456789|       1|
|     Jane Smith|    456 Elm St|jane.smith@exampl...|2345678901|       1|
|     Jane Smith|    456 Elm St|jane.smith2@examp...|5678901234|       2|
|       John Doe|   123 Main St|john.doe@example.com|1234567890|       1|
|       John Doe|   123 Main St|john.doe2@example...|8521469589|       2|
|Michael Johnson|   789 Oak Ave|michael.johnson@e...|3456789012|       1|
|Michael Johnson|   789 Oak Ave|michael.johnson2@...|9012345678|       2|
|    Sarah Davis| 333 Birch Ave|sarah.davis@examp...|7890123456|       1|
+---------------+--------------+------